# Using a deep CNN to directly classify SHL timeseries data

In [1]:
from pathlib import Path

MODEL_DIR = Path('models/shl-model-04-08-2021.h5')

In [2]:
# Define all datasets to train our model on

DATASET_DIRS = [
    Path('shl-dataset/challenge-2019-train_torso.zip'),
    Path('shl-dataset/challenge-2019-train_bag.zip'),
    Path('shl-dataset/challenge-2019-train_hips.zip'),
    Path('shl-dataset/challenge-2020-train_hand.zip'),
]

In [3]:
# Create scalers that we will fit on our data

from sklearn.preprocessing import StandardScaler

acc_scaler = StandardScaler()
mag_scaler = StandardScaler()
gyr_scaler = StandardScaler()

In [4]:
from tensorflow import keras

# Check that we can use our GPU, to not wait forever during training
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9358058155986742616,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 3215029044
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 7698256380391768418
 physical_device_desc: "device: 0, name: GeForce GTX 770, pci bus id: 0000:01:00.0, compute capability: 3.0"]

In [5]:
# Create our model

from tools.architectures import make_resnet
from tools.dataset import shl_dataset_label_order

if MODEL_DIR.is_file():
    # Load model for transfer learning
    model = keras.models.load_model(MODEL_DIR)
else:
    # Create model
    model = make_resnet(
        input_shape=[500, 3], # Magnitudes of ACC, MAG and Gyr, 500 data points each 
        output_classes=len(shl_dataset_label_order)
    )

In [7]:
import numpy as np

from tools.dataset import load_zipped_shl_dataset
from tools.export import export_standard_scaler

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tqdm.notebook import tqdm

for dataset_dir in DATASET_DIRS:
    # Load dataset from zip file into temporary directory
    dataset = load_zipped_shl_dataset(dataset_dir, tqdm=tqdm)

    # Fit and export scalers
    acc_mag_scaled = acc_scaler.fit_transform(dataset.acc_mag)
    mag_mag_scaled = mag_scaler.fit_transform(dataset.mag_mag)
    gyr_mag_scaled = gyr_scaler.fit_transform(dataset.gyr_mag)
    export_standard_scaler(acc_scaler, 'models/acc-scaler.json')
    export_standard_scaler(mag_scaler, 'models/mag-scaler.json')
    export_standard_scaler(gyr_scaler, 'models/gyr-scaler.json')

    # Prepare our training data
    X = np.stack([
        acc_mag_scaled,
        mag_mag_scaled, 
        gyr_mag_scaled,
    ], axis=2)
    y = np.eye(len(shl_dataset_label_order))[dataset.labels] # One Hot Encoding

    # Save some memory
    del dataset

    # Compute class weights for unbiased training
    y_integers = np.argmax(y, axis=1)
    class_weights = compute_class_weight('balanced', classes=np.unique(y_integers), y=y_integers)
    class_weights = dict(zip(np.unique(y_integers), class_weights)) # Keras adaption

    # Train model
    callbacks = [
        keras.callbacks.CSVLogger(f'{MODEL_DIR}.log', append=MODEL_DIR.is_file()),
        keras.callbacks.ModelCheckpoint(
            str(MODEL_DIR), save_best_only=True, monitor='val_loss'
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss', factor=0.5, patience=3, min_lr=0.0001
        ),
        
        keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1),
    ]
    history = model.fit(
        X, 
        y,
        epochs=10, 
        batch_size=32,
        class_weight=class_weights, 
        callbacks=callbacks,
        validation_split=0.1,
        verbose=1,
    )
    

Extracting shl-dataset\challenge-2019-train_torso.zip:   0%|          | 0/22 [00:00<?, ?it/s]

Acc_x Import Done
Acc_y Import Done
Acc_z Import Done
Acc_mag Import Done
Mag_x Import Done
Mag_y Import Done
Mag_z Import Done
Mag_mag Import Done
Gyr_x Import Done
Gyr_y Import Done
Gyr_z Import Done
Gyr_mag Import Done
Labels Import Done
Train on 176464 samples, validate on 19608 samples
Epoch 1/10
176464/176464 [==============================] - 592s 3ms/step - loss: 0.4659 - acc: 0.8050 - val_loss: 0.4512 - val_acc: 0.8341
Epoch 2/10
176464/176464 [==============================] - 583s 3ms/step - loss: 0.4192 - acc: 0.8265 - val_loss: 0.4764 - val_acc: 0.8260
Epoch 3/10
176464/176464 [==============================] - 582s 3ms/step - loss: 0.3907 - acc: 0.8396 - val_loss: 0.4328 - val_acc: 0.8452
Epoch 4/10
176464/176464 [==============================] - 583s 3ms/step - loss: 0.3715 - acc: 0.8489 - val_loss: 0.4779 - val_acc: 0.8310
Epoch 5/10
176464/176464 [==============================] - 584s 3ms/step - loss: 0.3586 - acc: 0.8539 - val_loss: 0.4603 - val_acc: 0.8378
Epoch 6/

Extracting shl-dataset\challenge-2019-train_bag.zip:   0%|          | 0/22 [00:00<?, ?it/s]

Acc_x Import Done
Acc_y Import Done
Acc_z Import Done
Acc_mag Import Done
Mag_x Import Done
Mag_y Import Done
Mag_z Import Done
Mag_mag Import Done
Gyr_x Import Done
Gyr_y Import Done
Gyr_z Import Done
Gyr_mag Import Done
Labels Import Done
Train on 176464 samples, validate on 19608 samples
Epoch 1/10
176464/176464 [==============================] - 601s 3ms/step - loss: 0.4101 - acc: 0.8320 - val_loss: 0.7500 - val_acc: 0.7375
Epoch 2/10
176464/176464 [==============================] - 603s 3ms/step - loss: 0.3373 - acc: 0.8628 - val_loss: 0.8609 - val_acc: 0.7360
Epoch 3/10
176464/176464 [==============================] - 604s 3ms/step - loss: 0.3099 - acc: 0.8742 - val_loss: 0.7413 - val_acc: 0.7547
Epoch 4/10
176464/176464 [==============================] - 605s 3ms/step - loss: 0.2919 - acc: 0.8829 - val_loss: 0.6959 - val_acc: 0.7722
Epoch 5/10
176464/176464 [==============================] - 587s 3ms/step - loss: 0.2776 - acc: 0.8884 - val_loss: 0.8166 - val_acc: 0.7590
Epoch 6/

Extracting shl-dataset\challenge-2019-train_hips.zip:   0%|          | 0/22 [00:00<?, ?it/s]

Acc_x Import Done
Acc_y Import Done
Acc_z Import Done
Acc_mag Import Done
Mag_x Import Done
Mag_y Import Done
Mag_z Import Done
Mag_mag Import Done
Gyr_x Import Done
Gyr_y Import Done
Gyr_z Import Done


MemoryError: 